In [126]:
import os
import pandas as pd
from sklearn import svm
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score
from collections import defaultdict
import os
import pandas as pd
from sklearn import svm
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

# Predicting Team to Certain Round in Tournament

In [127]:
# What do we do since they changed the seeding on the data with the introduction of the first-four

tournament_result_key = {"0" : "Didn't qualify", "1": "Round of 64", "2": "Round of 32", "4": "Sweet 16", "5": "Elite 8", "7": "Final 4",
                        "8": "Last 2", "9": "Champion"}
tournament_result_after_2010 = {"0" : "Didn't qualify", "1": "First Four", "2": "Round of 64", "3": "Round of 32", "4": "Sweet 16", "5": "Elite 8", "7": "Final 4",
                        "8": "Last 2", "9": "Champion"}

In [155]:
df = pd.read_csv("data/all years normalized.csv")

df = df.rename({'Tourament-Result': 'Tournament-Result'}, axis = 1)
df

/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_82269/2265776920.py:1: DtypeWarning: Columns (55,56,57,58,65,75,82,83,89,93) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/all years normalized.csv")


,Season,School,Conference,AP-Pre,AP-High,AP-Final,Tournament-Result,Tournament-Seed,G,W,...,3PAr Normalized,TS% Normalized,TRB% Normalized,AST% Normalized,STL% Normalized,BLK% Normalized,eFG% Normalized,TOV% Normalized,ORB% Normalized,FT/FGA Normalized
0,1992-93,air-force,WAC,30.0,30.0,30.0,0.0,20.0,28,9,...,0.446809,0.306667,0.503311,0.205047,#DIV/0!,0.841155,0.219355,0.403226,#DIV/0!,0.418502
1,1992-93,akron,MAC,30.0,30.0,30.0,0.0,20.0,26,8,...,0.440729,0.420000,0.582781,0.520505,#DIV/0!,0.693141,0.412903,0.483871,#DIV/0!,0.321586
2,1992-93,alabama,SEC,30.0,30.0,30.0,0.0,20.0,29,16,...,0.507599,0.580000,0.622517,0.167192,#DIV/0!,0.815884,0.574194,0.564516,#DIV/0!,0.453744
3,1992-93,alabama-state,SWAC,30.0,30.0,30.0,0.0,20.0,27,14,...,0.215805,0.686667,0.596026,0.271293,#DIV/0!,0.595668,0.606452,0.669355,#DIV/0!,0.603524
4,1992-93,alcorn-state,SWAC,30.0,30.0,30.0,0.0,20.0,27,7,...,0.094225,0.420000,0.370861,0.466877,#DIV/0!,0.754513,0.406452,0.588710,#DIV/0!,0.088106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8556,2017-18,wright-state,Horizon,30.0,30.0,30.0,1.0,14.0,35,25,...,0.332258,0.382022,0.613333,0.547112,0.471264,0.176871,0.363128,0.483146,0.505,0.600000
8557,2017-18,wyoming,MWC,30.0,30.0,30.0,0.0,20.0,33,20,...,0.661290,0.578652,0.300000,0.747720,0.356322,0.401361,0.541899,0.460674,0.02,0.862500
8558,2017-18,xavier,Big East,17.0,3.0,3.0,2.0,1.0,35,29,...,0.364516,0.786517,0.833333,0.686930,0.310345,0.306122,0.743017,0.359551,0.57,1.000000
8559,2017-18,yale,Ivy,30.0,30.0,30.0,0.0,20.0,31,16,...,0.570968,0.567416,0.480000,0.799392,0.45977,0.292517,0.597765,0.505618,0.36,0.381250


In [156]:
# Preprocessing to try and make it so it is game by game instead
# Note data already includes play in games (so no first four)
rounds = pd.read_csv("data/big_dance.csv")

rounds = rounds.dropna(how='all') #remove empty rows

# Obtain only years with practice data
rounds["Start_Year"] = rounds["Season"].str.split('-').str[0].astype(float)
rounds["End_Year"] = rounds["Season"].str.split('-').str[1].astype(float)

In [130]:
# Obtaining list of schools in order to normalize vectors
school_list = []
conference_list = []

for i in range(2,18):
    if i < 9:
        practice_season = "200{}-0{}".format(i, i + 1)
    elif i == 9:
        practice_season = "200{}-{}".format(i, i + 1)
    else:
        practice_season = "20{}-{}".format(i, i + 1)
    # Get a filter year
    practice_year = df.loc[df["Season"].str.contains(practice_season)]
    practice_year = practice_year.fillna(0)
                           
    # Get total list of schools to normalize data
    practice_schools = practice_year["School"].to_numpy()
    
    practice_conf = practice_year["Conference"].to_numpy()
    
    for school in practice_schools:
        if school not in school_list:
            school_list.append(school)
            
    for conf in practice_conf:
    
        if conf not in conference_list:
            if conf == 0:
                conf = "Ind"
            conference_list.append(conf)
            
school_list = list(set(school_list))
school_list.sort()
conference_list = list(set(conference_list))
conference_list.sort()

In [131]:
print(school_list)

['abilene-christian', 'air-force', 'akron', 'alabama', 'alabama-a&m', 'alabama-state', 'albany-(ny)', 'alcorn-state', 'american', 'appalachian-state', 'arizona', 'arizona-state', 'arkansas', 'arkansas-pine-bluff', 'arkansas-state', 'army', 'auburn', 'austin-peay', 'ball-state', 'baylor', 'belmont', 'bethune-cookman', 'binghamton', 'birmingham-southern', 'boise-state', 'boston-college', 'boston-university', 'bowling-green-state', 'bradley', 'brigham-young', 'brown', 'bryant', 'bucknell', 'buffalo', 'butler', 'cal-poly', 'cal-state-bakersfield', 'cal-state-fullerton', 'cal-state-northridge', 'california', 'campbell', 'canisius', 'centenary-(la)', 'central-arkansas', 'central-connecticut-state', 'central-florida', 'central-michigan', 'charleston-southern', 'charlotte', 'chattanooga', 'chicago-state', 'cincinnati', 'clemson', 'cleveland-state', 'coastal-carolina', 'colgate', 'college-of-charleston', 'colorado', 'colorado-state', 'columbia', 'connecticut', 'coppin-state', 'cornell', 'creigh

In [132]:
# Set up school vectors for predicting the winers without names
# And to check winners 
school_indices = dict((c, i) for i, c in enumerate(school_list))
indices_school = dict((i, c) for i, c in enumerate(school_list))

In [133]:
# Set up conference vectors for predicting the winners without names
conference_indices = dict((c, float(i)) for i, c in enumerate(conference_list))
indices_conference = dict((float(i), c) for i, c in enumerate(conference_list))
print(conference_indices)

{'A-10': 0.0, 'A-Sun': 1.0, 'AAC': 2.0, 'ACC': 3.0, 'AEC': 4.0, 'Big 12': 5.0, 'Big East': 6.0, 'Big Sky': 7.0, 'Big South': 8.0, 'Big Ten': 9.0, 'Big West': 10.0, 'CAA': 11.0, 'CUSA': 12.0, 'GWC': 13.0, 'Horizon': 14.0, 'Ind': 15.0, 'Ivy': 16.0, 'MAAC': 17.0, 'MAC': 18.0, 'MEAC': 19.0, 'MVC': 20.0, 'MWC': 21.0, 'Mid-Cont': 22.0, 'NEC': 23.0, 'OVC': 24.0, 'Pac-10': 25.0, 'Pac-12': 26.0, 'Patriot': 27.0, 'SEC': 28.0, 'SWAC': 29.0, 'Southern': 30.0, 'Southland': 31.0, 'Summit': 32.0, 'Sun Belt': 33.0, 'WAC': 34.0, 'WCC': 35.0}


In [144]:
# Setting up dictionary to connect teams to their data based on names
connecting_data = defaultdict()
tournament_teams = df


for i in range(2,18):
    if i < 9:
        practice_season = "200{}-0{}".format(i, i + 1)
    elif i == 9:
        practice_season = "200{}-{}".format(i, i + 1)
    else:
        practice_season = "20{}-{}".format(i, i + 1)
        
    year_data = defaultdict()
    year_teams = defaultdict()
    year = str(i+1)
    # Get a filter year
    practice_year = tournament_teams.loc[tournament_teams["Season"].str.contains(practice_season)]
    practice_year = practice_year.fillna(0)
    practice_year = practice_year.reset_index()

    practice_year = practice_year.drop(["Season"], axis =1)
    for ind in practice_year.index:
        
        team = practice_year["School"][ind]

        practice_year["School"][ind] = school_indices[team]
        
        if practice_year["Conference"][ind] == 0:

            practice_year["Conference"][ind] = conference_indices["Southland"]
        else:
            practice_year["Conference"][ind] = conference_indices[practice_year["Conference"][ind]]
        #print(ind, team)
        stats = practice_year.iloc[ind, :]
        # stats = stats.drop("School")
        
        
        drop_columns = [col for col in stats.index if "Normalized" not in col]
        drop_columns.remove("Conference")
        drop_columns.remove("School")
        
        drop_columns.append('Pace Normalized')
        drop_columns.append('ORtg Normalized')
        drop_columns.append('ORB% Normalized')
        drop_columns.append('STL% Normalized')
        drop_columns.append('3P Per Game Normalized')
        drop_columns.append('FG Per Game Normalized')
        drop_columns.append('FGA Per Game Normalized')
        drop_columns.append('MP Normalized')
        drop_columns.append('BLK% Normalized')
        drop_columns.append('TRB% Normalized')
        drop_columns.append('FT/FGA Normalized')
        drop_columns.append('FTA Per Game Normalized')
        drop_columns.append('FT Per Game Normalized')
        drop_columns.append('W-Conf Normalized')
        drop_columns.append('L-Conf Normalized')
        drop_columns.append('W-Away Normalized')
        drop_columns.append('L-Away Normalized')
        drop_columns.append('W-Home Normalized')
        drop_columns.append('L-Home Normalized')
        drop_columns.append('TOV% Normalized')
        drop_columns.append('FTr Normalized')
        drop_columns.append('BLK Per Game Normalized')
        drop_columns.append('PF Per Game Normalized')
        drop_columns.append('3PA Per Game Normalized')
        drop_columns.append('W-L% Normalized')
        drop_columns.append('TRB Per Game Normalized')
        drop_columns.append('eFG% Normalized')
        drop_columns.append('AST% Normalized')
        drop_columns.append('TS% Normalized')
        drop_columns.append('3PAr Normalized')
        drop_columns.append('FT% Normalized')
#         drop_columns.append('STL Per Game Normalized')
#         drop_columns.append('TOV Per Game Normalized')
#         drop_columns.append('ORB Per Game Normalized')
#         drop_columns.append('AST Per Game Normalized')
#         drop_columns.append('School')
#         drop_columns.append('Conference')
        drop_columns.append('W Normalized')
        drop_columns.append('L Normalized')
#         drop_columns.append('SRS Normalized')
#         drop_columns.append('SOS Normalized')
#         drop_columns.append('Pts-Tm. Per Game Normalized')
#         drop_columns.append('Pts-Opp. Per Game Normalized')
#         drop_columns.append('FG% Normalized')
#         drop_columns.append('3P% Normalized')
        
        stats = stats.drop(index=drop_columns)
        
        for stat in stats.index:
            if isinstance(stats[stat], str):
                try:
                    stats[stat] = float(stats[stat])
                except:
                    print(stat)
                    drop_columns.append(stat)
            
        
        
        year_data[team] = stats
        
    connecting_data[practice_season] = year_data


/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_82269/469128093.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practice_year["School"][ind] = school_indices[team]
/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_82269/469128093.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practice_year["Conference"][ind] = conference_indices[practice_year["Conference"][ind]]
/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_82269/469128093.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyd

/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_82269/469128093.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practice_year["School"][ind] = school_indices[team]
/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_82269/469128093.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practice_year["Conference"][ind] = conference_indices[practice_year["Conference"][ind]]
/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_82269/469128093.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyd

/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_82269/469128093.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practice_year["School"][ind] = school_indices[team]
/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_82269/469128093.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practice_year["Conference"][ind] = conference_indices[practice_year["Conference"][ind]]
/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_82269/469128093.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyd

SRS Normalized
SOS Normalized
SRS Normalized
SOS Normalized


/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_82269/469128093.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practice_year["School"][ind] = school_indices[team]
/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_82269/469128093.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practice_year["Conference"][ind] = conference_indices[practice_year["Conference"][ind]]
/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_82269/469128093.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyd

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [146]:
tourny_data = df.loc[df["Tournament-Result"] > 0]
practice_data = []
practice_targets = []
for i in range(2,17):
    
    if i < 9:
        practice_season = "200{}-0{}".format(i, i + 1)
    elif i == 9:
        practice_season = "200{}-{}".format(i, i + 1)
    else:
        practice_season = "20{}-{}".format(i, i + 1)
    
    practice_year = tourny_data.loc[tourny_data["Season"].str.contains(practice_season)]
    
    practice_year_data = practice_year[["School", "Tournament-Result"]]
    
    for ind in practice_year_data.index:
        team = practice_year["School"][ind]
        result = practice_year["Tournament-Result"][ind]
        
        team_data = connecting_data[practice_season][team]
        practice_data.append(team_data)
        practice_targets.append(result)
    

practice_data = np.array(practice_data)
print(practice_data.shape)
practice_targets = np.array(practice_targets)

practice_data = practice_data.reshape(-1, 12)
practice_targets = practice_targets.reshape(-1, 1)

(977, 12)


In [147]:
print(practice_data)
print(practice_targets)

[[3 28.0 0.78021148 ... 0.656676897 0.556450201 0.641427538]
 [10 25.0 0.970166163 ... 0.961987347 0.738895939 0.715909091]
 [11 25.0 0.811555891 ... 0.855475567 0.466529188 0.668487762]
 ...
 [355 8.0 0.542200957 ... 0.305860345 0.4707852 0.43487703]
 [356 9.0 0.88784689 ... 0.370720542 0.502904094 0.219011771]
 [360 6.0 0.799808612 ... 0.460750213 0.387873636 0.384189139]]
[[1.]
 [5.]
 [2.]
 [4.]
 [1.]
 [1.]
 [4.]
 [2.]
 [2.]
 [1.]
 [1.]
 [1.]
 [4.]
 [1.]
 [1.]
 [4.]
 [1.]
 [2.]
 [2.]
 [1.]
 [2.]
 [2.]
 [1.]
 [8.]
 [5.]
 [1.]
 [2.]
 [1.]
 [7.]
 [4.]
 [1.]
 [5.]
 [1.]
 [1.]
 [2.]
 [1.]
 [4.]
 [5.]
 [2.]
 [1.]
 [1.]
 [4.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [9.]
 [7.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [4.]
 [2.]
 [1.]
 [5.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [2.]
 [9.]
 [1.]
 [2.]
 [7.]
 [1.]
 [1.]
 [1.]
 [1.]
 [8.]
 [2.]
 [4.]
 [1.]
 [5.]
 [2.]
 [1.]
 [1.]
 [1.]
 [2.]
 [2.]
 [2.]
 [1.]
 [2.]
 [1.]
 [1.]
 [2.]
 [4.]
 [2.]
 [1.]
 [7.]
 [2.

In [148]:
X_train, X_test, y_train, y_test = train_test_split(practice_data, practice_targets, test_size=0.3, random_state=42)

#normalize, come back to this later
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

model = GaussianNB()
model.fit(X_train, y_train)

# model.fit(X_train_scaled, y_train)

predictions = model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

print(predictions)


Accuracy: 0.43197278911564624
Classification Report:
              precision    recall  f1-score   support

         1.0       0.66      0.60      0.63       105
         2.0       0.39      0.40      0.39        92
         3.0       0.00      0.00      0.00        18
         4.0       0.33      0.45      0.38        40
         5.0       0.24      0.29      0.26        21
         7.0       0.11      0.14      0.12         7
         8.0       0.00      0.00      0.00         8
         9.0       0.18      0.67      0.29         3

    accuracy                           0.43       294
   macro avg       0.24      0.32      0.26       294
weighted avg       0.42      0.43      0.42       294

[5. 4. 1. 2. 2. 1. 5. 1. 2. 2. 4. 1. 2. 1. 2. 4. 2. 2. 2. 1. 1. 1. 2. 4.
 1. 5. 4. 1. 9. 7. 9. 2. 7. 4. 5. 5. 1. 1. 2. 4. 4. 1. 1. 4. 2. 4. 5. 1.
 2. 1. 5. 5. 4. 1. 1. 2. 1. 1. 2. 2. 1. 4. 2. 1. 2. 2. 4. 2. 1. 2. 1. 1.
 2. 5. 5. 4. 9. 2. 2. 7. 4. 2. 2. 2. 1. 8. 1. 1. 4. 1. 4. 1. 2. 4. 1. 1.
 4. 

/Users/eddieloyd/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [149]:
test_season = "2017-18"

In [150]:
test_data = []
test_targets = []
test_year = tourny_data.loc[tourny_data["Season"].str.contains(test_season)]
    
test_year_data = test_year[["School", "Tournament-Result"]]

for ind in test_year_data.index:
    team = test_year["School"][ind]
    result = test_year["Tournament-Result"][ind]

    team_data = connecting_data[test_season][team]
    test_data.append(team_data)
    test_targets.append(result)

test_data = np.array(test_data)
test_targets = np.array(test_targets)

test_data = test_data.reshape(-1, 12)
test_targets = test_targets.reshape(-1, 1)

In [151]:
# Predicting model on following year
predictions = np.array(model.predict(test_data))

In [152]:
predictions_to_round = {"0": 0, "1": 1, "2": 2, "3":3, "4": 4, "5" : 5, "6": 7, "7": 8, "8": 9}

In [153]:
for i, team in enumerate(test_year_data["School"]):
    print(team, predictions[i])
    

alabama 2.0
arizona 4.0
arkansas 2.0
auburn 2.0
bucknell 1.0
buffalo 2.0
butler 2.0
cal-state-fullerton 1.0
cincinnati 2.0
clemson 2.0
college-of-charleston 1.0
creighton 4.0
davidson 2.0
duke 9.0
florida 2.0
florida-state 2.0
georgia-state 1.0
gonzaga 2.0
houston 2.0
iona 1.0
kansas 5.0
kansas-state 2.0
kentucky 2.0
lipscomb 1.0
loyola-(il) 1.0
marshall 2.0
maryland-baltimore-county 1.0
miami-(fl) 2.0
michigan 4.0
michigan-state 4.0
missouri 2.0
montana 1.0
murray-state 1.0
nc-state 2.0
nevada 2.0
new-mexico-state 1.0
north-carolina 5.0
ohio-state 4.0
oklahoma 2.0
pennsylvania 1.0
providence 2.0
purdue 5.0
radford 1.0
rhode-island 2.0
san-diego-state 2.0
seton-hall 4.0
south-dakota-state 2.0
st.-bonaventure 2.0
stephen-f.-austin 1.0
syracuse 2.0
tcu 4.0
tennessee 7.0
texas 2.0
texas-a&m 2.0
texas-southern 1.0
texas-tech 7.0
unc-greensboro 1.0
villanova 5.0
virginia 4.0
virginia-tech 4.0
west-virginia 4.0
wichita-state 4.0
wright-state 1.0
xavier 4.0


In [154]:
accuracy = accuracy_score(predictions, test_targets)
print(accuracy)


0.390625
